**COURSE: PRDL/MLLB**

**PROJECT: Deep Learning**

**TEACHER: Luis Hernández Gómez**

**AUTHORS: MARONE Mamadou / RACHIDI Inass**

**NOTEBOOK: CNN & LSTM**

# SETUP

## INSTALLING MODULES

In [ ]:
%%capture
!pip install tensorflow
!pip install tqdm

## IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Load and Prepare Data

In [ ]:
# os.chdir(r"C:\Users\maron\OneDrive\02-Documents\00.ETUDES\00.ECOLE_D_INGE\00.CYCLE_ING_FORMATION_INIT\00.3EME_ANNEE_INIT\00.A_COURS\00.PRDL\06.PROJECTS")

In [ ]:
df_cleaned = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PROJET_DL_MLLB/DATA/CLEANED/corpus_cleaned.csv")
df_cleaned.head(3)

,category,title,body,text
0,ARTS & CULTURE,model agenc enabl sexual predat year former ag...,octob carolyn kramer receiv disturb phone call...,model agenc enabl sexual predat year former ag...
1,ARTS & CULTURE,actor jeff hiller talk bright color bold patte...,week talk actor jeff hiller hit broadway play ...,actor jeff hiller talk bright color bold patte...
2,ARTS & CULTURE,new yorker cover put trump hole racist comment,new yorker take presid donald trump ask u woul...,new yorker cover put trump hole racist comment...


In [ ]:
# Encode the labels
label_encoder = LabelEncoder()
df_cleaned['encoded_labels'] = label_encoder.fit_transform(df_cleaned['category'])

In [ ]:
# Split the data into features and labels
X = df_cleaned['text']
y = df_cleaned['encoded_labels']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Tokenize and Pad Text Data

In [ ]:
# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words = max_words, oov_token = '<OOV>')
tokenizer.fit_on_texts(X_train)

In [ ]:
# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Pad sequences for equal length
max_length = 500
X_train_padded = pad_sequences(X_train_sequences, maxlen = max_length, padding = 'post', truncating = 'post')
X_test_padded = pad_sequences(X_test_sequences, maxlen = max_length, padding = 'post', truncating = 'post')

# MODEL

In [ ]:
model = Sequential()

embedding_dim = 128
# Embedding layer
model.add(Embedding(input_dim = max_words, output_dim = embedding_dim, input_length = max_length))

In [ ]:
num_filters = 256
filter_size = 3

# CNN layer
model.add(Conv1D(filters = num_filters, kernel_size = filter_size, activation = 'relu'))
model.add(MaxPooling1D(pool_size = 2))

In [ ]:
# LSTM layer
model.add(LSTM(256, activation = 'tanh', recurrent_activation = 'sigmoid'))

In [ ]:
# Output layer
num_classes = len(label_encoder.classes_)
model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
# Compile the model
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Display the model summary
model.summary()

# TRAINING

In [ ]:
epochs = 40
batch_size = 10

from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y_train_one_hot = to_categorical(y_train, num_classes = num_classes)
y_test_one_hot = to_categorical(y_test, num_classes = num_classes)

W notice that the batch size affect a lot the accuracy :
    - 0.45 of accuracy with 100
    - 0.50 of accuracy with 50
    - 0.58 of accuracy with 30
Number of epoch:
    - with batch size of 30 - epoch of 30 : 62%
    

In [ ]:
# Fit the model with one-hot encoded labels
history = model.fit(X_train_padded, y_train_one_hot, epochs = epochs, batch_size = batch_size, validation_data = (X_test_padded, y_test_one_hot))

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss values
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Notice:
- with the **default activation** functions:
    - The model underfits(0.17 accuracy) when we use **Adam** as optimizer algorithm. When We have changed it to **RMSprop**, we got 0.23 of accuracy and the underfitting decreases.
- with the activation functions **activation='tanh'** and **recurrent_activation='sigmoid'** :
    - the Adam perform better than the RMSprop (17% Vs 22%)
    - We have also tried the optimizer **Nadam**(17%)

## EVALUATION

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_padded, y_test_one_hot)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

  1/172 [..............................] - ETA: 18s - loss: 0.0349 - accuracy: 1.0000

172/172 [==============================] - 14s 83ms/step - loss: 0.0152 - accuracy: 0.9960
Test Loss: 0.015177085064351559
Test Accuracy: 0.9960007071495056


In [ ]:
# Evaluate the model on the test set
y_pred = model.predict_classes(X_test_padded)

# Calculate additional metrics
accuracy = accuracy_score(y_test_one_hot, y_pred)
conf_matrix = confusion_matrix(y_test_one_hot, y_pred)
classification_rep = classification_report(y_test_one_hot, y_pred)

print("Test Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

# Extract precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test_one_hot, y_pred, average='weighted')

print("Weighted Precision:", precision)
print("Weighted Recall:", recall)
print("Weighted F1 Score:", f1_score)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()